# SCENIC Protocol - Case study - Mouse brain data set


__Author:__ Bram Van de Sande

__Date:__ 6 AUG 2019

__Outline:__ Acquistion and cleaning of selected scRNAseq data sets.

_Experiments:_

| Accession ID | Cancer type | 
| ------------- | ----------- | 
| GSE60361 | Mouse brain |

In [1]:
import os, glob, re, pickle
from functools import partial
from collections import OrderedDict
import operator as op
from cytoolz import compose

import pandas as pd
import seaborn as sns
import numpy as np
import scanpy as sc
import anndata as ad
import matplotlib as mpl
import matplotlib.pyplot as plt

from pyscenic.export import export2loom, add_scenic_metadata
from pyscenic.utils import load_motifs
from pyscenic.transform import df2regulons
from pyscenic.aucell import aucell
from pyscenic.binarization import binarize
from pyscenic.rss import regulon_specificity_scores
from pyscenic.plotting import plot_binarization, plot_rss

In [17]:
# Set maximum number of jobs
sc.settings.njobs = 32

In [38]:
import pandas as pd
# confirm that gene names are present in cistarget databases
ranking_feather = pd.read_feather("/work/rsenovilla/kss/resources/mm10_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather")
loom=ad.read_loom("/work/rsenovilla/kss/mp2.loom")
ex_matrix_og = loom.to_df().T
overlap_values = ex_matrix_og.index[pd.Series(ex_matrix_og.index).isin(ranking_feather.columns)].unique()


In [37]:
ex_matrix = ex_matrix_og.loc[overlap_values, :]

In [41]:
#save matrix
ex_matrix.to_csv(COUNTS_QC_MTX_FNAME)

In [9]:
RESOURCES_FOLDERNAME = "/work/rsenovilla/scenic/kss/"
RESULTS_FOLDERNAME = "/work/rsenovilla/kss/results/"
FIGURES_FOLDERNAME = "/work/rsenovilla/kss/results/"
AUXILLIARIES_FOLDERNAME = "/work/rsenovilla/kss/resources/"

In [10]:
# Ranking databases. Downloaded from cisTargetDB: https://resources.aertslab.org/cistarget/
RANKING_DBS_FNAMES = list(map(lambda fn: os.path.join("/work/rsenovilla/kss/resources/", fn),
                       ['mm10_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather',
                       'mm10_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather']))
# Motif annotations. Downloaded from cisTargetDB: https://resources.aertslab.org/cistarget/
MOTIF_ANNOTATIONS_FNAME = os.path.join("/work/rsenovilla/scenic/cistarget", 'motifs-v10nr_clust-nr.mgi-m0.001-o0.0.tbl')

In [11]:
DATASET_ID = 'mp2'

In [12]:
# Downloaded expression matrix from GEO on 13 JUN 2019 (https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE60361).
COUNTS_MTX_FNAME = os.path.join(RESULTS_FOLDERNAME, 'mp2-Expression.txt')
# Downloaded metadata from http://linnarssonlab.org/cortex/ on 13 JUN 2019
METADATA_FNAME = os.path.join(RESULTS_FOLDERNAME, 'mp2_metadata.txt')

MM_TFS_NAME = "/work/rsenovilla/scenic/data_me9/allTFs_mm.txt"

# Output file
COUNTS_QC_MTX_FNAME = os.path.join(RESULTS_FOLDERNAME, 'mp2.qc.counts.csv')
ADJACENCIES_FNAME = "/work/rsenovilla/kss/results/adj.tsv"
MOTIFS_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.motifs.csv'.format(DATASET_ID))
REGULONS_DAT_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.regulons.dat'.format(DATASET_ID))
AUCELL_MTX_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.auc.csv'.format(DATASET_ID))
BIN_MTX_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.bin.csv'.format(DATASET_ID))
THR_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.thresholds.csv'.format(DATASET_ID))
ANNDATA_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.h5ad'.format(DATASET_ID))
LOOM_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.loom'.format(DATASET_ID))

In [7]:
#Create list of M. musculus transcription factors.
pd_motifs = pd.read_csv("/work/rsenovilla/scenic/data_me9/allTFs_mm.txt", sep='\t', index_col=0, header = None)

In [8]:
mm_tfs = pd_motifs.index.unique()
len(mm_tfs)

1860

In [9]:
with open(MM_TFS_FNAME, 'wt') as f:
    f.write('\n'.join(mm_tfs) + '\n')

In [74]:
loom_path = "/work/rsenovilla/kss/mp2.loom"

In [13]:
adata = ad.read_loom(loom_path)
adata.var_names_make_unique()
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
adata.raw = adata #Store non-log transformed data as raw. This data can be used via the use_raw parameters available for many functions.
df_counts_qc = adata.to_df()
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
adata

AnnData object with n_obs × n_vars = 3634 × 17283
    obs: 'Batch', 'CellType', 'Clusters_Ant_0.5', 'Condition', 'FerCellType', 'G2M.Score', 'GABA_Clusters', 'GLUT_Clusters', 'Glutamatergic.mean', 'Interneurons.mean', 'Named_Subclustering', 'Original_CyclePhase', 'Original_G2M.Score', 'Original_S.Score', 'Phase', 'RNA_snn_res.0.05', 'RNA_snn_res.0.1', 'RNA_snn_res.0.2', 'RNA_snn_res.0.3', 'RNA_snn_res.0.4', 'RNA_snn_res.0.5', 'RNA_snn_res.0.6', 'RNA_snn_res.0.8', 'RNA_snn_res.1', 'RNA_snn_res.1.2', 'RNA_snn_res.1.8', 'RNA_snn_res.2.4', 'S.Score', 'Selected_Cells', 'integrated_snn_res.0.5', 'nCount_RNA', 'nCount_SCT', 'nFeature_RNA', 'nFeature_SCT', 'obs_names', 'old.ident', 'orig.ident', 'origen', 'percent.mt', 'sample_name', 'seurat_clusters', 'n_genes'
    var: 'features', 'var_names', 'n_cells'
    uns: 'log1p'

In [25]:
df_counts_qc.to_csv(COUNTS_QC_MTX_FNAME)

In [ ]:
# sc.pp.highly_variable_genes(adata)
sc.pl.highly_variable_genes(adata)

In [55]:
adata = adata[:, adata.var['highly_variable']]

In [56]:
sc.tl.pca(adata, svd_solver='arpack')

/work/rsenovilla/miniconda3/envs/pyscenice/lib/python3.10/site-packages/scanpy/preprocessing/_pca.py:325: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  adata.obsm["X_pca"] = X_pca


In [57]:
sc.tl.tsne(adata)

In [ ]:
sc.set_figure_params(frameon=False, dpi=150, fontsize=8)
sc.pl.tsne(adata, color=['Named_Subclustering'], 
           title=['GSE60361 - Cell types'], ncols=2, use_raw=False)

### STEP 1: Network inference based on GRNBoost2 from CLI

For this step the CLI version of SCENIC is used. This step can be deployed on an High Performance Computing system.

_Output:_ List of adjacencies between a TF and its targets stored in `ADJACENCIES_FNAME`.

In [38]:
from pyscenic.cli import arboreto_with_multiprocessing

In [55]:
#dask gave problems, so we opted for arboreto instead of pyscenic grn (below)
!arboreto_with_multiprocessing.py \
    /work/rsenovilla/kss/mp2.loom \
    /work/rsenovilla/scenic/data_me9/allTFs_mm.txt \
    --method grnboost2 \
    --output results/mp2-adj.tsv \
    --num_workers 40 \
    --seed 777

Loaded expression matrix of 3634 cells and 21297 genes in 2.4384098052978516 seconds...
Loaded 1860 TFs...
starting grnboost2 using 40 processes...
 25%|████████▏                        | 5264/21297 [3:53:48<11:42:29,  2.63s/it]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



!pyscenic grn {loom_path} {MM_TFS_NAME} -o {ADJACENCIES_FNAME} --num_workers 32

```
2019-06-13 17:32:45,044 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.
2019-06-13 17:33:07,766 - pyscenic.cli.pyscenic - INFO - Inferring regulatory networks.
preparing dask client
parsing input
creating dask graph
32 partitions
computing dask graph
not shutting down client, client was created externally
finished
2019-06-13 18:04:23,356 - pyscenic.cli.pyscenic - INFO - Writing results to file.
```

### STEP 2-3: Regulon prediction aka cisTarget from CLI

For this step the CLI version of SCENIC is used. This step can be deployed on an High Performance Computing system.

_Output:_ List of adjacencies between a TF and its targets stored in `MOTIFS_FNAME`.

In [14]:
DBS_PARAM = ' '.join(RANKING_DBS_FNAMES)

In [15]:
ADJACENCIES_FNAME="/work/rsenovilla/kss/results/mp2-adj.tsv"

In [59]:
!pyscenic ctx {ADJACENCIES_FNAME} {DBS_PARAM} \
            --annotations_fname {MOTIF_ANNOTATIONS_FNAME} \
            --expression_mtx_fname {loom_path} \
            --mask_dropouts \
            --output {MOTIFS_FNAME} \
            --num_workers 40


2024-07-14 20:37:39,536 - pyscenic.cli.pyscenic - INFO - Creating modules.

2024-07-14 20:37:41,778 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2024-07-14 20:37:47,567 - pyscenic.utils - INFO - Calculating Pearson correlations.

2024-07-14 20:37:48,054 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [True].

2024-07-14 20:37:55,960 - pyscenic.utils - INFO - Creating modules.

2024-07-14 20:39:23,678 - pyscenic.cli.pyscenic - IN

```
2019-06-14 09:49:42,740 - pyscenic.cli.pyscenic - INFO - Creating modules.
2019-06-14 09:49:45,193 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.
2019-06-14 09:52:42,315 - pyscenic.cli.pyscenic - INFO - Loading databases.
2019-06-14 09:52:42,316 - pyscenic.cli.pyscenic - INFO - Calculating regulons.
2019-06-14 10:18:06,172 - pyscenic.cli.pyscenic - INFO - Writing results to file.
```

In [60]:
df_motifs = load_motifs(MOTIFS_FNAME)

In [67]:
df_motifs

AUC       NES  \
TF     MotifID                                                     
Ahdc1  metacluster_188.6                      0.096294  3.121497   
Arid3a tfdimers__MD00270                      0.074937  3.309930   
       tfdimers__MD00152                      0.079619  3.663798   
Arx    metacluster_184.32                     0.113715  3.334312   
       metacluster_184.7                      0.113163  3.307452   
...                                                ...       ...   
Zfp143 metacluster_34.4                       0.073285  3.126991   
Zfp180 metacluster_3.4                        0.074566  3.077104   
Zfp202 metacluster_3.7                        0.102346  3.184092   
Zfp273 metacluster_39.1                       0.113466  3.415047   
Zfp786 kznf__ZNF786_Imbeault2017_RP_ChIP-seq  0.105918  3.053168   

                                              MotifSimilarityQvalue  \
TF     MotifID                                                        
Ahdc1  metacluster_188.6                               2.001250e-13   
Arid3a tfdimers__MD00270                               1.501390e-06   
       tfdimers__MD00152                               7.834240e-07   
Arx    metacluster_184.32                              4.510380e-06   
       metacluster_184.7                               2.774820e-06   
...                                                             ...   
Zfp143 metacluster_34.4                                0.000000e+00   
Zfp180 metacluster_3.4                                 7.911120e-06   
Zfp202 metacluster_3.7                                 6.749830e-07   
Zfp273 metacluster_39.1                                0.000000e+00   
Zfp786 kznf__ZNF786_Imbeault2017_RP_ChIP-seq           0.000000e+00   

                                              OrthologousIdentity  \
TF     MotifID                                                      
Ahdc1  metacluster_188.6                                 0.953576   
Arid3a tfdimers__MD00270                                 0.814503   
       tfdimers__MD00152                                 0.803661   
Arx    metacluster_184.32                                1.000000   
       metacluster_184.7                                 0.957447   
...                                                           ...   
Zfp143 metacluster_34.4                                  1.000000   
Zfp180 metacluster_3.4                                   0.676692   
Zfp202 metacluster_3.7                                   0.795632   
Zfp273 metacluster_39.1                                  0.616766   
Zfp786 kznf__ZNF786_Imbeault2017_RP_ChIP-seq             0.649936   

                                                                                     Annotation  \
TF     MotifID                                                                                    
Ahdc1  metacluster_188.6                      gene is orthologous to ENSG00000126705 in H. s...   
Arid3a tfdimers__MD00270                      motif similar to tfdimers__MD00454 ('M00377_fo...   
       tfdimers__MD00152                      gene is orthologous to ENSG00000116017 in H. s...   
Arx    metacluster_184.32                     motif similar to cisbp__M00467 ('Arx[gene ID: ...   
       metacluster_184.7                      gene is orthologous to ENSG00000004848 in H. s...   
...                                                                                         ...   
Zfp143 metacluster_34.4                                              gene is directly annotated   
Zfp180 metacluster_3.4                        motif similar to kznf__ZNF180_Imbeault2017_OM_...   
Zfp202 metacluster_3.7                        gene is orthologous to ENSG00000166261 in H. s...   
Zfp273 metacluster_39.1                       gene is orthologous to ENSG00000197013 in H. s...   
Zfp786 kznf__ZNF786_Imbeault2017_RP_ChIP-seq  gene is orthologous to ENSG00000197362 in H. s...   

                                                        

### STEP 4: Cellular enrichment aka AUCell

__REGULON CREATION__

Regulons can easily be created from this list of enriched motifs via `pyscenic.transform.df2regulons`. Here we provide an auxilliary function to carefully select the enriched motifs that contribute to the regulons.

In [70]:
def derive_regulons(motifs, db_names=('mm10_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings',
                       'mm10_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.rankings')):
    motifs.columns = motifs.columns.droplevel(0)

    def contains(*elems):
        def f(context):
            return any(elem in context for elem in elems)
        return f

    # For the creation of regulons we only keep the 10-species databases and the activating modules. We also remove the
    # enriched motifs for the modules that were created using the method 'weight>50.0%' (because these modules are not part
    # of the default settings of modules_from_adjacencies anymore.
    motifs = motifs[
        np.fromiter(map(compose(op.not_, contains('weight>50.0%')), motifs.Context), dtype=bool) & \
        np.fromiter(map(contains(*db_names), motifs.Context), dtype=bool) & \
        np.fromiter(map(contains('activating'), motifs.Context), dtype=bool)]

    # We build regulons only using enriched motifs with a NES of 3.0 or higher; we take only directly annotated TFs or TF annotated
    # for an orthologous gene into account; and we only keep regulons with at least 10 genes.
    regulons = list(filter(lambda r: len(r) >= 10, df2regulons(motifs[(motifs['NES'] >= 3.0) 
                                                                      & ((motifs['Annotation'] == 'gene is directly annotated')
                                                                        | (motifs['Annotation'].str.startswith('gene is orthologous to')
                                                                           & motifs['Annotation'].str.endswith('which is directly annotated for motif')))
                                                                     ])))
    
    # Rename regulons, i.e. remove suffix.
    return list(map(lambda r: r.rename(r.transcription_factor), regulons))

In [71]:
regulons = derive_regulons(df_motifs)

Create regulons from a dataframe of enriched features.
Additional columns saved: []


In [72]:
regulons

[Regulon(name='Arx', gene2weight=frozendict.frozendict({'Ccnd2': 1.870284515233496, 'Dlx1as': 1.580290460113092, 'Pax6': 2.781558804856424, 'Sp9': 3.4713539739843355, 'Sox2': 1.9282159837183015, 'Dlx2': 4.732778289211643, 'Sox1': 2.9252743421727883, 'Dlx6os1': 1.742980564001405, 'Gad1': 1.1409145044351356, 'Mafb': 1.4439938827247545, 'Efhd2': 1.0803902852580844, 'Pou3f4': 1.2763700133763944, 'Dlx6': 2.5433852580075462, 'Dclk2': 1.3830499320021754, 'Gli2': 2.801699710907429, 'Map3k20': 1.0650980372230476, 'Gm13889': 1.533811338997427, 'Erbb4': 1.4407994081298368, 'Pfn2': 1.459170447513812, 'Gad2': 2.105868631001162, 'Vav2': 0.5058548954570844, 'Dlx1': 4.601221156468147, 'Nrxn3': 1.928589565537035, 'Rgs8': 0.8719534682266883, 'Arx': 1.0, 'Cdca7': 2.9352514477450704, 'Nin': 0.8200388838047026, 'Cdca7l': 0.6113037146227973, 'Gng2': 0.7595949650137567, 'Enox1': 0.977379766106453, 'Rgs16': 1.1289233075313438, 'Tox3': 1.0340252794113776, 'Maf': 0.4247121116803888, 'Grip1': 0.520168258112534, 

In [110]:
# Pickle these regulons.
with open(REGULONS_DAT_FNAME, 'wb') as f:
    pickle.dump(regulons, f)

__AUCELL__

In [83]:
#df counts can be obtained with adata.to_df()
%%time
auc_mtx = aucell(df_counts.T, regulons, num_workers=32)
auc_mtx.to_csv(AUCELL_MTX_FNAME)

CPU times: user 6.64 s, sys: 4.29 s, total: 10.9 s
Wall time: 16.3 s


In [84]:
auc_mtx

Regulon,Arx,Atf1,BC025920,Bcl11a,Chd2,Cnot3,Dlx1,Dlx2,E2f1,E2f2,...,Zfp143,Zfp273,Zfp3,Zfp467,Zfp518a,Zfp647,Zfp711,Zfp800,Zfp955b,Zscan12
Cell,,,,,,,,,,,,,,,,,,,,,
R1_AAACCCAAGATGGGCT-1,0.004848,0.000000,0.000000,0.055541,0.000000,0.009707,0.029422,0.018111,0.013117,0.000000,...,0.000000,0.004681,0.000000,0.000073,0.000000,0.000000,0.00000,0.000000,0.000000,0.0
R1_AAACCCACAAACTAGA-1,0.026391,0.006784,0.007760,0.044363,0.012682,0.014327,0.039953,0.042464,0.017663,0.002723,...,0.000000,0.000000,0.000000,0.001047,0.000000,0.000000,0.00000,0.000000,0.028165,0.0
R1_AAACCCACAGGTGGAT-1,0.000000,0.015873,0.000000,0.072550,0.009579,0.018868,0.014922,0.008907,0.000000,0.005070,...,0.038779,0.000000,0.043727,0.014367,0.000000,0.000000,0.00000,0.008684,0.000000,0.0
R1_AAACCCACAGTTGCGC-1,0.067279,0.008376,0.005278,0.045946,0.032143,0.017905,0.061572,0.047567,0.035861,0.009871,...,0.000000,0.000000,0.000000,0.005920,0.021616,0.000000,0.00000,0.000000,0.000000,0.0
R1_AAACCCATCACGACTA-1,0.020370,0.001224,0.000000,0.059060,0.000000,0.017294,0.051814,0.046339,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.006078,0.000000,0.000000,0.01205,0.014790,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R2_TTTGGAGTCTTCTCAA-1,0.297973,0.023065,0.000000,0.056399,0.000000,0.013921,0.304779,0.325354,0.014375,0.002473,...,0.000000,0.000000,0.000000,0.004109,0.000000,0.000000,0.00000,0.067699,0.000000,0.0
R2_TTTGGTTCAACCAACT-1,0.005101,0.001326,0.005903,0.112495,0.000000,0.017455,0.016202,0.011099,0.003706,0.000000,...,0.001150,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.071696,0.000000,0.0
R2_TTTGGTTCACCGGCTA-1,0.037079,0.002271,0.000000,0.074734,0.000000,0.012795,0.076300,0.075001,0.015166,0.004158,...,0.041503,0.000000,0.000000,0.007263,0.000000,0.000000,0.01301,0.000000,0.030555,0.0


```
CPU times: user 10.5 s, sys: 4.41 s, total: 14.9 s
Wall time: 14 s
```

In [25]:
auc_mtx = pd.read_csv(AUCELL_MTX_FNAME, index_col=0)

__AUCELL + tSNE PROJECTION__

We add all metadata derived from SCENIC to the `scanpy.AnnData` object.

In [87]:
add_scenic_metadata(adata, auc_mtx, regulons)

AnnData object with n_obs × n_vars = 3634 × 21297
    obs: 'Batch', 'CellType', 'Clusters_Ant_0.5', 'Condition', 'FerCellType', 'G2M.Score', 'GABA_Clusters', 'GLUT_Clusters', 'Glutamatergic.mean', 'Interneurons.mean', 'Named_Subclustering', 'Original_CyclePhase', 'Original_G2M.Score', 'Original_S.Score', 'Phase', 'RNA_snn_res.0.05', 'RNA_snn_res.0.1', 'RNA_snn_res.0.2', 'RNA_snn_res.0.3', 'RNA_snn_res.0.4', 'RNA_snn_res.0.5', 'RNA_snn_res.0.6', 'RNA_snn_res.0.8', 'RNA_snn_res.1', 'RNA_snn_res.1.2', 'RNA_snn_res.1.8', 'RNA_snn_res.2.4', 'S.Score', 'Selected_Cells', 'integrated_snn_res.0.5', 'nCount_RNA', 'nCount_SCT', 'nFeature_RNA', 'nFeature_SCT', 'obs_names', 'old.ident', 'orig.ident', 'origen', 'percent.mt', 'sample_name', 'seurat_clusters', 'Regulon(Arx)', 'Regulon(Atf1)', 'Regulon(BC025920)', 'Regulon(Bcl11a)', 'Regulon(Chd2)', 'Regulon(Cnot3)', 'Regulon(Dlx1)', 'Regulon(Dlx2)', 'Regulon(E2f1)', 'Regulon(E2f2)', 'Regulon(E2f3)', 'Regulon(E2f6)', 'Regulon(E2f7)', 'Regulon(Elf2)', '

In [88]:
sc.tl.tsne(adata, use_rep='X_aucell')

In [108]:
regulons

Index(['Regulon(Arx)', 'Regulon(Atf1)', 'Regulon(BC025920)', 'Regulon(Bcl11a)',
       'Regulon(Chd2)', 'Regulon(Cnot3)', 'Regulon(Dlx1)', 'Regulon(Dlx2)',
       'Regulon(E2f1)', 'Regulon(E2f2)', 'Regulon(E2f3)', 'Regulon(E2f6)',
       'Regulon(E2f7)', 'Regulon(Elf2)', 'Regulon(Elk1)', 'Regulon(Fos)',
       'Regulon(Fosb)', 'Regulon(Fosl2)', 'Regulon(Foxm1)', 'Regulon(Foxo1)',
       'Regulon(Gabpb1)', 'Regulon(Gm3854)', 'Regulon(Gsx1)', 'Regulon(Gtf2i)',
       'Regulon(Gtf2ird1)', 'Regulon(Hes1)', 'Regulon(Hivep2)',
       'Regulon(Irf2)', 'Regulon(Irf3)', 'Regulon(Jun)', 'Regulon(Junb)',
       'Regulon(Jund)', 'Regulon(Klf11)', 'Regulon(Klf7)', 'Regulon(Klf9)',
       'Regulon(Lef1)', 'Regulon(Lhx2)', 'Regulon(Lhx6)', 'Regulon(Mafk)',
       'Regulon(Mef2a)', 'Regulon(Mlx)', 'Regulon(Mybl1)', 'Regulon(Nfib)',
       'Regulon(Nfix)', 'Regulon(Nkx2-2)', 'Regulon(Nr2f6)', 'Regulon(Nr4a1)',
       'Regulon(Nr4a2)', 'Regulon(Nr6a1)', 'Regulon(Pbx3)', 'Regulon(Pknox1)',
       'Regulo

In [ ]:
sc.set_figure_params(frameon=False, dpi=150, fontsize=8)
for i in adata.var.T.index[2:]: 
 sc.pl.tsne(adata, color=['Named_Subclustering', i], 
           title=['GSE60361 - Cell types', i], ncols=2, use_raw=False)

In [ ]:
sc.set_figure_params(frameon=False, dpi=150, fontsize=8)
sc.pl.tsne(adata, color=['Named_Subclustering', 'Regulon(Nfib)'], 
           title=['GSE60361 - Cell types', 'Nfib'], ncols=2, use_raw=False)

In [ ]:
sc.set_figure_params(frameon=False, dpi=150, fontsize=8)
sc.pl.tsne(adata, color=['level1class', 'sex', 'age', 'Gad1'], 
           title=['GSE60361 - Cell types', 'Sex', 'Age', 'Gad1'], ncols=2)

In [114]:
adata.write_h5ad("/work/rsenovilla/kss/results/output_mp2.h5ad")

In [4]:
adata.obs.to_csv("/work/rsenovilla/kss/results/output_mp2_obs.csv") #to generae figures in R with original UMAP